In [1]:
# Import the necessary libraries
import pandas as pd
import quantstats as qs
from openbb_terminal.sdk import openbb

Forecasting Toolkit is disabled. To use the Forecasting features please install the toolkit following the instructions here: https://my.openbb.co/app/sdk/installation

Portfolio Optimization Toolkit is disabled. To use the Optimization features please install the toolkit following the instructions here: https://my.openbb.co/app/sdk/installation

In [3]:
# Fetch stock data for a list of stocks using the OpenBB terminal.
# We have selected FAANG stocks for this analysis.
tickers = ["META", "AAPL", "AMZN", "NFLX", "GOOG"]
data = openbb.economy.index(
    tickers, 
    start_date="2010-01-01", 
    end_date="2022-03-28"
)

In [4]:
# Define the lookback period for momentum calculation.
# Here, we're setting it to approximately 12 months
# (with an average of 21 trading days per month).
lookback = 12 * 21

# We want to select and trade the top 3 assets based on their momentum.
traded_count = 3

# Define initial cash and set the cash to this value.
initial_cash = 100000
cash = initial_cash

# Initialize our portfolio as an empty dictionary.
portfolio = {}

# Initialize a pandas Series to keep track of our portfolio's value over time.
portfolio_value = pd.Series(index=data.index)

In [5]:
def rebalance(data, current_date):
    """
    Determine which assets to hold based on momentum.
    """
    # Calculate momentum for each asset using a rolling mean and then percentage change.
    momentum = data.rolling(lookback).mean().pct_change()

    # Get the momentum values for the specific current_date.
    momentum_on_date = momentum.loc[current_date]

    # Sort the assets by momentum in descending order.
    sorted_by_momentum = momentum_on_date.sort_values(ascending=False)

    # Take the top assets based on the number defined by 'traded_count'.
    selected_assets = sorted_by_momentum.index[:traded_count]

    return selected_assets

In [6]:
# Iterate over each date in our dataset.
for date in data.index:
    # Calculate the current value of the portfolio on the given date.
    current_value = 0
    for ticker in tickers:
        current_value += portfolio.get(ticker, 0) * data.loc[date, ticker]

    # Add any uninvested cash to the current portfolio value.
    current_value += cash

    # Store the current value in the portfolio_value series.
    portfolio_value.loc[date] = current_value

    # If it's the first day of the month, we rebalance our portfolio.
    if date.day == 1:
        # First, sell all our current holdings.
        for ticker in tickers:
            cash += portfolio.get(ticker, 0) * data.loc[date, ticker]

        # Reset the portfolio to an empty state.
        portfolio = {}

        # Decide which assets to buy based on their momentum.
        selected_assets = rebalance(data, date)

        # Equally divide our cash among the selected assets.
        cash_per_asset = cash / traded_count

        # Purchase the selected assets.
        for asset in selected_assets:
            portfolio[asset] = cash_per_asset / data.loc[date, asset]
            cash -= cash_per_asset

In [7]:
returns = portfolio_value.pct_change()

In [8]:
qs.reports.metrics(
    returns,
    mode="full"
)

                           Strategy
-------------------------  ----------
Start Period               2012-05-21
End Period                 2022-03-25
Risk-Free Rate             0.0%
Time in Market             100.0%

Cumulative Return          878.71%
CAGR﹪                     17.34%

Sharpe                     0.99
Prob. Sharpe Ratio         99.89%
Smart Sharpe               0.92
Sortino                    1.41
Smart Sortino              1.31
Sortino/√2                 1.0
Smart Sortino/√2           0.93
Omega                      1.19

Max Drawdown               -35.75%
Longest DD Days            476
Volatility (ann.)          27.25%
Calmar                     0.49
Skew                       -0.34
Kurtosis                   4.79

Expected Daily %           0.09%
Expected Monthly %         1.94%
Expected Yearly %          23.04%
Kelly Criterion            8.86%
Risk of Ruin               0.0%
Daily Value-at-Risk        -2.72%
Expected Shortfall (cVaR)  -2.72%

Max Consecutive Wins    